In [1]:
import pandas as pd
import requests
import time
import snowflake.connector
from snowflake.connector import SnowflakeConnection

In [2]:
# Your copied token from Swagger or any Raptive endpoint
AUTH_TOKEN = "YOUR_AUTH_TOKEN"

In [3]:
SNOWFLAKE_CONNECTION_PARAMS = {
      "account": "SNOWFLAKE_ACCOUNT",
      "user": "SNOWFLAKE_USER",
      "password": "SNOWFLAKE_PASSWORD",
      "warehouse": "SNOWFLAKE_WAREHOUSE",
      "schema": "SNOWFLAKE_SCHEMA",
  }

In [4]:
def create_snowflake_connection(
):
  """
  Creates a connection to snowflake
  """
  ctx = snowflake.connector.connect(
      **(SNOWFLAKE_CONNECTION_PARAMS),
  )
  return ctx

def get_snowflake_data(
  ctx: SnowflakeConnection,
  sql_script: str,
  **script_params
):
  cur = ctx.cursor()
  try:
      cur.execute(sql_script, script_params)
      df_temp = cur.fetch_pandas_all()
      df_temp.columns = df_temp.columns.str.lower()
  except Exception as e:
      print(f"Error in executing the SQL script: {e}")
      cur.close()
      ctx.close()
      raise
  finally:
      cur.close()
      ctx.close()
  return df_temp

In [5]:
# Get Sites that need to be backfilled 
site_query = """
SELECT site_id
from analytics.adthrive.site_extended
where status = 'Active'
"""
ctx = create_snowflake_connection()
df_sites = get_snowflake_data(ctx, site_query)

In [7]:
# Your list of sites (replace with real site group identifiers or domains)
site_ids = df_sites['site_id'].tolist()

HEADERS = {
    "Authorization": AUTH_TOKEN,
    "Content-Type": "application/json"
}

# Endpoint from Swagger
API_URL = "https://api.raptive.com/api/v2/gscBackfill/group"

# Customize the date range to backfill
def get_payload(site):
    return {
            "siteId": site,
            "periodStartDate": "2025-04-23",
            "periodEndDate": "2025-04-23",
            "status": "PENDING"
          }

In [ ]:
# Make requests
for site in site_ids:
    payload = get_payload(site)
    print(f"\nSending payload for {site}:\n{payload}")
    
    try:
        response = requests.post(API_URL, headers=HEADERS, json=payload)
        print(f"Response [{response.status_code}]")
        
        if response.status_code == 201 or response.status_code == 200:
            print(f"✅ Success: {site}")
        else:
            print(f"❌ Failed: {site}")
    except Exception as e:
        print(f"⚠️ Error for {site}: {str(e)}")

    time.sleep(1)



In [14]:
response.json()

[{'backfillGroupId': 'bfbb9baa-5d47-49dd-8f79-bf8a6adc16d8',
  'backfillEntries': [{'id': 85582,
    'siteId': '67af8e7805195f08435b95f1',
    'periodStartDate': '2025-04-23',
    'periodEndDate': '2025-04-23',
    'dimension': 'PAGE',
    'status': 'PENDING',
    'numFailedAttempts': 0},
   {'id': 85583,
    'siteId': '67af8e7805195f08435b95f1',
    'periodStartDate': '2025-04-23',
    'periodEndDate': '2025-04-23',
    'dimension': 'DEVICE',
    'status': 'PENDING',
    'numFailedAttempts': 0},
   {'id': 85584,
    'siteId': '67af8e7805195f08435b95f1',
    'periodStartDate': '2025-04-23',
    'periodEndDate': '2025-04-23',
    'dimension': 'KEYWORD',
    'status': 'PENDING',
    'numFailedAttempts': 0},
   {'id': 85585,
    'siteId': '67af8e7805195f08435b95f1',
    'periodStartDate': '2025-04-23',
    'periodEndDate': '2025-04-23',
    'dimension': 'DISCOVER',
    'status': 'PENDING',
    'numFailedAttempts': 0}]}]